# Explorando convolução no Keras usando o Theano

O objetivo deste notebook é de explorar a convolução que é implementada pelo Theano através do Keras.

A convolução é a operação essencial das redes convolucionais. É uma operação demorada e a sua implementação na GPU acelera bastante o seu processamento. A implementação é feita no Theano utilizando GPU. Se a GPU não for encontrada, a implementação roda na CPU.

[A guide to convolution arithmetic for deep
learning by Vincent Dumoulin and Francesco Visin, 2016](https://arxiv.org/pdf/1603.07285.pdf)


In [1]:
import numpy as np
import keras
from keras.models import Sequential
from keras.utils.np_utils import convert_kernel



Using Theano backend.


# Uma rede com uma única camada convolucional

O objetivo deste experimento é explorar numericamente a convolução implementada na rede
convolucional do Keras/Theano. Para isso iremos criar uma rede convolucional com apenas
uma camada de convolução. A sua predição será o resultado da rede, que neste caso será apenas uma convolução.

Uma convolução pode ser visto como uma média móvel ponderada sobre a imagem. A equação matemática da convolução é dada por:

$$ \boldsymbol{Y} = \boldsymbol{X} \ast \boldsymbol{W} $$

onde $x$ é a imagem de entrada, $y$ é o resultado da convolução e $w$ é chamado de peso,
núcleo, máscara, entre outros nomes. A convolução é a implementação de filtro linear
invariante à translação. Este filtro é totalmente especificado pelo núcleo (array $w$) que
contém os parâmetros do filtro linear (da rede convolucional) que precisam ser treinados.

A figura a seguir ilustra uma convolução da imagem verde de entrada, utilizando o núcleo da convolução (kernel) em amarelo, deslizando sobre a imagem verde. O resultado da convolução é a imagem rosa, de tamanho ligeiramente menor, devido ao processamento na borda da imagem.

![](http://mourafiq.com/images/posts/convolution_schematic.gif)

Esta próxima figura ilustra a convolução quando a imagem de saída (em verde) é igual às
dimensões da imagem de entrada (em azul). Neste caso, a imagem de entrada é artificialmente
aumentada com valores zeros. Este processo de aumentar a imagem de entrada com zeros é 
denominado *zero padding*.

![](https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/same_padding_no_strides.gif)

## Organização da imagem de entrada

As convoluções do Theano foram projetadas principalmente para tratar imagens ou sons. Assim, a organização dos arrays da imagem de entrada, de saída e dos pesos são feitos de forma específica. Iremos ilustrar apenas as convoluções utilizando imagem de entrada em nível de cinza e coloridas com 3 bandas. Diferentemente da organização utilizada nos classificadores usuais (não convolucionais), a organização da matriz $\boldsymbol{X}$ de entrada é com $n$ 
linhas e $k$ colunas, onde as colunas são as *features*. O shape de $\boldsymbol{X}$ é então

$$ (n,k)$$.
 
No caso de imagens, como elas têm
natureza bidimensional, a relação de vizinhança entre os pixels é fundamental, principalmente
para que os pesos dos núcleo abranjam uma região bidimensional da imagem. Assim, as *features* passam agora a terem duas dimensões: são $k$ pixels organizados em (img_height, img_width). Desta forma o array passa a ter uma nova dimensão: ($n$, image_height, image_width). Numa 
imagem em nível de cinza temos apenas um valor para representar o pixel, que é sua intensidade, variando de escuro a claro. Quando a imagem é colorida, são necessários 3 valores
para cada pixel. No Theano, este 3 valores são organizados em uma dimensão adicional, denominada canal (*channel*) ou banda. A imagem colorida é organizada em shape:(bandas, img_height, img_width). Desta forma o shape da imagem de entrada deve ter 4 dimensões:

$\boldsymbol{X}: $` (samples, channels, img_height, img_width)`

## Organização dos pesos (kernel) da convolução

Os pesos do kernel da convolução para tratar imagens devem ser 2D, quando a imagem
de entrada for em nível de cinza e 3 bandas 2D para imagens coloridas. Assim, a organização
do shape do kernel é $(bands, I_{height}, I_{width})$. Entretanto, nas arquiteturas das redes
convolucionais, é usual definir vários filtros (vários kernels), para que a rede tenha um
maior número de parâmetros a serem ajustados. Assim, o shape dos pesos devem ter 4 dimensões:

$\boldsymbol{W}:$ (nb_filters, channels, img_height, img_width)

## Organização da imagem de saída (filtrada)

Tanto a imagem de entrada como o *kernel* possuem quatro dimensões, porém os significados
da primeira dimensão é diferente, pois na imagem X a primeira dimensão é o número de amostras,
no *kernel*, a primeira dimensão é o número de filtros de saída. Já as 3 dimensões restantes
são relativas à imagem, em cinza (channels=1) ou colorida (channels=3). Uma das especificidades da convolução do Theano é que a convolução de uma imagem 2D por um kernel 2D
resulta numa imagem 2D, porém se a imagem for colorida, o resultado da convolução de uma
imagem 2D com 3 bandas por um kernel 2D com 3 bandas será um imagem 2D (de uma banda apenas).
Isto dá espaço para colocar o número de imagens resultantes dos número de filtros numa
dimensão e com a dimensão do número de amostras, ficamos com dimensão 4 na imagem de saída:

$\boldsymbol{Y}: $` (samples, filtros, img_height, img_width)`



## Definição da convolução usando Keras

A implementação da convolução utiliza um modelo sequencial do Keras com `Convolution2D`.
Obtemos o resultado da rede pela função `predict`:

In [32]:
# 4-D convolution
def conv4d(x,w,border_mode='same'):
    assert x.ndim == 4
    assert w.ndim == 4
    nb_filters,wd,wh,ww = w.shape
    samples,channels,xh,xw = x.shape
    model = Sequential([
                Convolution2D(nb_filters, wh, ww, 
                              input_shape=(channels, xh, xw),
                              weights=[W], 
                              border_mode=border_mode, bias=False, dim_ordering='th')
    ])
    y = model.predict(x,batch_size=1)
    return y


## Primeiro caso, resposta ao impulso

Quando a imagem de entrada é apenas um 1 rodeado de vários zeros, o resultado da convolução
é o próprio kernel posicionado no local do 1. Esta é uma forma usual para se verificar se
a convolução está funcionando. Assim, utilizaremos uma imagem cinza 3x5 com um "1" no centro.
O kernel é de 2 linhas e 3 colunas. O resultado é uma imagem 3x5 com o kernel posicionado
no local do 1 da imagem de entrada.

In [51]:
np.set_printoptions(suppress=True, precision=3)

x = np.array([[0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0],
              [0,0,0,1,0,0,0],
              [0,0,0,0,0,0,0],
              [0,0,0,0,0,0,0]]).reshape(1,1,5,7)
W = np.array([[1.,2.,3.],
              [4.,5.,6.]]).reshape(1,1,2,3)

y = conv4d(x,W)

print 'x:', x.shape
print x
print 'y:', y.shape
print y
print 'kernel'
w = model.layers[0].get_weights()
print w



x: (1, 1, 5, 7)
[[[[0 0 0 0 0 0 0]
   [0 0 0 0 0 0 0]
   [0 0 0 1 0 0 0]
   [0 0 0 0 0 0 0]
   [0 0 0 0 0 0 0]]]]
y: (1, 1, 5, 7)
[[[[ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]
   [ 0.  0.  1.  2.  3.  0.  0.]
   [ 0.  0.  4.  5.  6.  0.  0.]
   [ 0.  0.  0.  0.  0.  0.  0.]]]]
kernel
[array([[[[ 1.,  2.,  3.],
         [ 4.,  5.,  6.]]]], dtype=float32)]


## Reproduzindo o exemplo da figura animada acima

In [29]:
W = np.array([[1,0,1],
              [0,1,0],
              [1,0,1]]).reshape(1,1,3,3)
x = np.array([[1,1,1,0,0],
              [0,1,1,1,0],
              [0,0,1,1,1],
              [0,0,1,1,0],
              [0,1,1,0,0]]).reshape(1,1,5,5)
y = conv4d(x,W,'valid')
print 'x:', x.shape
print x
print 'y:', y.shape
print y


x: (1, 1, 5, 5)
[[[[1 1 1 0 0]
   [0 1 1 1 0]
   [0 0 1 1 1]
   [0 0 1 1 0]
   [0 1 1 0 0]]]]
y: (1, 1, 3, 3)
[[[[ 4.  3.  4.]
   [ 2.  4.  3.]
   [ 2.  3.  4.]]]]


## 4 amostras de imagem cinza 3x5, 1 filtro com kernel de 3x3 

Neste caso, 
- shape da entrada, x:(4,1,3,5)
- shape dos pesos W:(1, 1, 3,3). 
- shape de saída será y:(4,1,3,5)

In [38]:
W = np.array([[1,0,1],
              [0,1,0],
              [1,0,1]]).reshape(1,1,3,3)
# criando 4 amostras de imagens
x = np.array([[0,0,0,0,0],
              [0,0,1,0,0],
              [0,0,0,0,0]]).reshape(1,1,3,5) * np.arange(4).reshape(4,1,1,1)
y = conv4d(x,W,'same')
print 'x:', x.shape
print x
print 'y:', y.shape
print y


x: (4, 1, 3, 5)
[[[[0 0 0 0 0]
   [0 0 0 0 0]
   [0 0 0 0 0]]]


 [[[0 0 0 0 0]
   [0 0 1 0 0]
   [0 0 0 0 0]]]


 [[[0 0 0 0 0]
   [0 0 2 0 0]
   [0 0 0 0 0]]]


 [[[0 0 0 0 0]
   [0 0 3 0 0]
   [0 0 0 0 0]]]]
y: (4, 1, 3, 5)
[[[[ 0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.]
   [ 0.  0.  0.  0.  0.]]]


 [[[ 0.  1.  0.  1.  0.]
   [ 0.  0.  1.  0.  0.]
   [ 0.  1.  0.  1.  0.]]]


 [[[ 0.  2.  0.  2.  0.]
   [ 0.  0.  2.  0.  0.]
   [ 0.  2.  0.  2.  0.]]]


 [[[ 0.  3.  0.  3.  0.]
   [ 0.  0.  3.  0.  0.]
   [ 0.  3.  0.  3.  0.]]]]


## 2 filtros com kernel de 3x3, 1 amostra de imagem cinza 3x5

Neste caso, 
- shape da entrada, x:(1,1,3,5)
- shape dos pesos W:(2, 1, 3,3). 
- shape de saída será y:(1,2,3,5)


In [39]:
W = np.array([[[1,0,1],
               [0,1,0],
               [1,0,1]],
              [[0,1,0],
               [1,2,1],
               [0,1,0]]]).reshape(2,1,3,3)
x = np.array([[0,0,0,0,0],
              [0,0,1,0,0],
              [0,0,0,0,0]]).reshape(1,1,3,5)
y = conv4d(x,W,'same')
print 'x:', x.shape
print x
print 'y:', y.shape
print y


x: (1, 1, 3, 5)
[[[[0 0 0 0 0]
   [0 0 1 0 0]
   [0 0 0 0 0]]]]
y: (1, 2, 3, 5)
[[[[ 0.  1.  0.  1.  0.]
   [ 0.  0.  1.  0.  0.]
   [ 0.  1.  0.  1.  0.]]

  [[ 0.  0.  1.  0.  0.]
   [ 0.  1.  2.  1.  0.]
   [ 0.  0.  1.  0.  0.]]]]


## 1 filtro com kernel de 3x3 e 3 canais, 1 amostra de imagem 3x5 RGB (3 canais)
Neste caso, como a imagem de entrada possui 3 bandas, para guardar as bandas Red, Green e Blue, o kernel também precisa ter 3 bandas pois são necessários pesos para cada banda de cor.
- shape da entrada, x:(1,3,3,5)
- shape dos pesos W:(1, 3, 3,3). 
- shape de saída será y:(1,2,3,5)


In [50]:
W = np.array([[[1,0,1],
               [0,1,0],
               [1,0,1]],
              [[0,1,0],
               [1,1,1],
               [0,1,0]],
              [[0,1,0],
               [0,1,0],
               [0,1,0]]]).reshape(1,3,3,3)
x = np.array([[[0,0,0,0,0],
               [0,1,0,0,0],
               [0,0,0,0,0]],
              [[0,0,0,0,1],
               [0,0,1,0,0],
               [0,0,0,0,0]],
              [[0,0,0,0,0],
               [0,0,0,1,0],
               [0,0,0,0,0]]]).reshape(1,3,3,5)
y = conv4d(x,W,'same')
print 'x:', x.shape
print x
print 'W:', W.shape
print W
print 'y:', y.shape
print y


 x: (1, 3, 3, 5)
[[[[0 0 0 0 0]
   [0 1 0 0 0]
   [0 0 0 0 0]]

  [[0 0 0 0 1]
   [0 0 1 0 0]
   [0 0 0 0 0]]

  [[0 0 0 0 0]
   [0 0 0 1 0]
   [0 0 0 0 0]]]]
W: (1, 3, 3, 3)
[[[[1 0 1]
   [0 1 0]
   [1 0 1]]

  [[0 1 0]
   [1 1 1]
   [0 1 0]]

  [[0 1 0]
   [0 1 0]
   [0 1 0]]]]
y: (1, 1, 3, 5)
[[[[ 1.  0.  2.  2.  1.]
   [ 0.  2.  1.  2.  1.]
   [ 1.  0.  2.  1.  0.]]]]


## Exercícios

### Teórico

1. Supondo uma imagem de entrada 100 x 100 e um kernel 3 x 3. Utilizando-se o border_mode como válido, a imagem de saída será de 99 x 99. Se fosse utilizar uma rede densa, com entrada de 100 x 100 e saída 99 x 99, quantos parâmetros seriam necessários para ser treinados? E no caso da rede convolucional? Qual é o fator de redução?

### Práticos

1. Trocar o parâmetro ``border_mode`` para ``same`` e veja a diferença.
2. Rodar a convolução para uma imagem de cinza
3. Rodar a convolução para uma imagem colorida RGB
4. Rodar a convolução para gerar várias bandas de saída como nas redes convolucionais
5. Comparar o tempo de execução desta convolução com outras implementações: openCV por exemplo